In [20]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [21]:
csv_path = 'https://raw.githubusercontent.com/jpvegah/Report/main/Madrid.txt'

In [22]:
mynewdf2 = pd.read_csv(csv_path, header=0)

In [23]:
mynewdf2[0:5]

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,11,Centro,Palacio,40.416514,-3.713708
1,12,Centro,Embajadores,40.409869,-3.702972
2,13,Centro,Cortes,40.415226,-3.697128
3,14,Centro,Justicia,40.423660,-3.696469
4,15,Centro,Universidad,40.407078,-3.709642


In [24]:
mynewdf2.columns

Index(['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [25]:
mynewdf2.dtypes

Postal Code       int64
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [26]:
print('The dataframe has {} Boroughs and {} Neighborhoods.'.format(
        len(mynewdf2['Borough'].unique()),
        mynewdf2.shape[0]
    )
)
mynewdf2['Borough'].unique()

The dataframe has 21 Boroughs and 131 Neighborhoods.


array(['Centro', 'Arganzuela', 'Retiro', 'Salamanca', 'Chamartin',
       'Tetuan', 'Chamberi', 'Fuencarral-El Pardo', 'Moncloa-Aravaca',
       'Latina', 'Carabanchel', 'Usera', 'Puente de Vallecas',
       'Moratalaz', 'Ciudad Lineal', 'Hortaleza', 'Villaverde',
       'Villa de Vallecas', 'Vicalvaro', 'San Blas-Canillejas', 'Barajas'],
      dtype=object)

In [27]:
#Create a Map of Madrid with Neighborhoods superimposed using latitude and longitude values
#Passing Madrid coordinates manually
latitude=40.41831
longitude=-3.70275

map_madrid = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(mynewdf2['Latitude'], mynewdf2['Longitude'], mynewdf2['Borough'], mynewdf2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
map_madrid

In [28]:
#Next, start utilizing the Foursquare API
CLIENT_ID = 'ET5MEYPN14RDTKJNHY4VVC0Q12R114CIOWGYVZ14FBSK1V32' # your Foursquare ID
CLIENT_SECRET = 'SUJMUIQO4X24ZKIPS31ZJFA12FJTFIC11SJQXSGZG4ZMRLSV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ET5MEYPN14RDTKJNHY4VVC0Q12R114CIOWGYVZ14FBSK1V32
CLIENT_SECRET:SUJMUIQO4X24ZKIPS31ZJFA12FJTFIC11SJQXSGZG4ZMRLSV


In [29]:
#Let´s explore the first neighborhood
mynewdf2.loc[0, 'Neighborhood']
#Get the Neighborhood's Latitude and Longitude
neighborhood_latitude = mynewdf2.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = mynewdf2.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = mynewdf2.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Palacio are 40.416514023873745, -3.7137077865943455.


In [34]:
#Now let´s get the top 100 venues in 500m distance
#Let´s create the GET request
#Import Latitude and Longitude Values
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id=ET5MEYPN14RDTKJNHY4VVC0Q12R114CIOWGYVZ14FBSK1V32&client_secret=SUJMUIQO4X24ZKIPS31ZJFA12FJTFIC11SJQXSGZG4ZMRLSV&ll=40.41831,-3.70275&oauth_token=&v=20180605&radius=500&limit=100'
url

'https://api.foursquare.com/v2/venues/explore?client_id=ET5MEYPN14RDTKJNHY4VVC0Q12R114CIOWGYVZ14FBSK1V32&client_secret=SUJMUIQO4X24ZKIPS31ZJFA12FJTFIC11SJQXSGZG4ZMRLSV&ll=40.41831,-3.70275&oauth_token=&v=20180605&radius=500&limit=100'

In [35]:
#Send the GET request ans examine the result
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6018563837a6ef7de4da76c5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Sol',
  'headerFullLocation': 'Sol, Madrid',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 80,
  'suggestedBounds': {'ne': {'lat': 40.4228100045, 'lng': -3.696850330083829},
   'sw': {'lat': 40.413809995499996, 'lng': -3.708649669916171}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bddaac30ee3a59370242eb0',
       'name': 'Club del Gourmet Corte Ingles',
       'location': {'address': 'C. Preciados, 3',
        'lat': 40.41749665225322,
        'lng': -3.704686028127798,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.41749665

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
#Clean the json and structure into a Pandas Dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Club del Gourmet Corte Ingles,Gourmet Shop,40.417497,-3.704686
1,LUSH,Cosmetics Shop,40.419012,-3.704898
2,Hotel Liabeny,Hotel,40.418742,-3.703996
3,Kiko store,Cosmetics Shop,40.420128,-3.702317
4,TAKOS,Mexican Restaurant,40.418938,-3.703748


In [38]:
#How many venues returned by FourSquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

80 venues were returned by Foursquare.


In [39]:
## Next Explore Neighbourhoods in Madrid

In [40]:
#Create a function to repeat the same process to all Neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
               
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
#Run the above function on each Neighbourhood and create the Dataframe madrid_venues
madrid_venues = getNearbyVenues(names=mynewdf2['Neighborhood'],
                                   latitudes=mynewdf2['Latitude'],
                                   longitudes=mynewdf2['Longitude'])

Palacio
Embajadores
Cortes
Justicia
Universidad
Sol
Imperial
Acacias
Chopera
Legazpi
Delicias
Palos de Moguer
Atocha
Pacifico
Adelfas
Estrella
Ibiza
Jeronimos
Nino Jesus
Recoletos
Goya
Fuente del Berro
Guindalera
Lista
Castellana
El Viso
Prosperidad
Ciudad Jardin
Hispanoamerica
Nueva Espana
Castilla
Bellas Vistas
Cuatro Caminos
Castillejos
Almenara
Valdeacederas
Berruguete
Gaztambide
Arapiles
Trafalgar
Almagro
Rios Rosas
Vallehermoso
El Pardo
Fuentelarreina
Penagrande
Pilar
La Paz
Valverde
Mirasierra
El Goloso
Casa de Campo
Arguelles
Ciudad Universitaria
Valdezarza
Valdemarin
El Plantio
Aravaca
Los Carmenes
Puerta del Angel
Lucero
Aluche
Campamento
Cuatro Vientos
Las Aguilas
Comillas
Opanel
San Isidro
Vista Alegre
Puerta Bonita
Buenavista
Abrantes
Orcasitas
Orcasur
San Fermin
Almendrales
Moscardo
Zofio
Pradolongo
Entrevias
San Diego
Palomeras Bajas
Palomeras Sureste
Portazgo
Numancia
Pavones
Horcajo
Marroquina
Media Legua
Fontarron
Vinateros
Ventas
Pueblo Nuevo
Quintana
Concepcion
San 

In [43]:
#Size of the resulting Dataframe
print(madrid_venues.shape)
madrid_venues.head()


(3439, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Palacio,40.416514,-3.713708,Palacio Real de Madrid,40.417940,-3.714259,Palace
1,Palacio,40.416514,-3.713708,Santa Iglesia Catedral de Santa María la Real ...,40.415767,-3.714516,Church
2,Palacio,40.416514,-3.713708,Plaza de La Almudena,40.416320,-3.713777,Plaza
3,Palacio,40.416514,-3.713708,Plaza de Oriente,40.418326,-3.712196,Plaza
4,Palacio,40.416514,-3.713708,Zuccaru,40.417179,-3.711674,Ice Cream Shop


In [44]:
#Find how many unique categories can be curated from all the returned venues
print('There are {} unique categories.'.format(len(madrid_venues['Venue Category'].unique())))

There are 277 unique categories.


In [45]:
##Analyze each Neighborhood
# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
madrid_onehot['Neighborhood'] = madrid_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]

madrid_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
#New dataframe size
madrid_onehot.shape

(3439, 277)

In [47]:
##Let´s group rows by Neighborhood and by taking the mean of the frequency of the ocurrence of each category
madrid_grouped = madrid_onehot.groupby('Neighborhood').mean().reset_index()
madrid_grouped


,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Arcade,Arepa Restaurant,...,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Abrantes,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Acacias,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Adelfas,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aeropuerto,0.0,0.0,0.0,0.058824,0.294118,0.058824,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alameda de Osuna,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Ventas,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,Villaverde Alto,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,Vinateros,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,Vista Alegre,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
#Confirm new size
madrid_grouped.shape


(127, 277)

In [49]:
#Let´s print each neighbourhood along with the top 5 most common venues
num_top_venues = 5

for hood in madrid_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = madrid_grouped[madrid_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----Abrantes----
                  venue  freq
0                  Park  0.29
1         Metro Station  0.14
2           Pizza Place  0.14
3  Gym / Fitness Center  0.14
4    Athletics & Sports  0.14


----Acacias----
                venue  freq
0  Spanish Restaurant  0.11
1                 Bar  0.09
2         Supermarket  0.06
3    Tapas Restaurant  0.06
4         Pizza Place  0.06


----Adelfas----
                  venue  freq
0    Spanish Restaurant  0.08
1           Supermarket  0.08
2         Grocery Store  0.08
3                   Bar  0.05
4  Fast Food Restaurant  0.05


----Aeropuerto----
                venue  freq
0      Airport Lounge  0.29
1  Spanish Restaurant  0.12
2         Coffee Shop  0.12
3     Border Crossing  0.06
4         Men's Store  0.06


----Alameda de Osuna----
                 venue  freq
0           Smoke Shop  0.10
1         Cocktail Bar  0.05
2  Fried Chicken Joint  0.05
3          Pizza Place  0.05
4   Spanish Restaurant  0.05


----Almagro----
           

In [50]:
#Put into a Pandas Dataframe
#First use function to classify venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
#Now let´s create a dataframe and display the top 10 venues for each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = madrid_grouped['Neighborhood']

for ind in np.arange(madrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abrantes,Park,Pizza Place,Athletics & Sports,Metro Station,Fast Food Restaurant,Gym / Fitness Center,Women's Store,Falafel Restaurant,Escape Room,Event Space
1,Acacias,Spanish Restaurant,Bar,Tapas Restaurant,Supermarket,Pizza Place,Asian Restaurant,Gym / Fitness Center,Gym,Park,Peruvian Restaurant
2,Adelfas,Spanish Restaurant,Grocery Store,Supermarket,Bar,Fast Food Restaurant,Coffee Shop,Bookstore,Soccer Field,Skate Park,Breakfast Spot
3,Aeropuerto,Airport Lounge,Spanish Restaurant,Coffee Shop,Sporting Goods Shop,Breakfast Spot,Border Crossing,Airport Gate,Airport Service,Fast Food Restaurant,Duty-free Shop
4,Alameda de Osuna,Smoke Shop,Pizza Place,Bakery,Cocktail Bar,Plaza,Chinese Restaurant,Restaurant,Scenic Lookout,Shop & Service,Spanish Restaurant


In [52]:
## Cluster Neighbourhoods
# Run k-means to cluster the neighbourhoods into 12 clusters
# set number of clusters
kclusters = 12

madrid_grouped_clustering = madrid_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([ 7,  5,  5,  5,  5,  5,  5,  0,  5,  7,  7,  5, 11,  5,  5,  0,  5,
        5,  5,  5,  7,  8,  0,  5,  9,  5,  3,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  2,  5,  5,  5,  5,  7,  5,  5,  5,
       10,  5,  5,  5,  7,  5,  0,  0,  5,  5,  5,  0,  5,  5,  5,  7,  5,
        5,  0,  7,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  0,  5,  7,  7,  0,  5,  5,  0,  5,  6,
        5,  5,  0,  5,  4,  5,  5,  5,  5,  5,  1,  5,  5,  0,  5,  0,  0,
        0,  5,  5, 10,  5,  5,  5,  0], dtype=int32)

In [53]:
#Let´s create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

madrid_merged = mynewdf2

# merge madrid_grouped with mynewdf2 to add latitude/longitude for each neighborhood
madrid_merged = madrid_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

madrid_merged.head() # check the last columns!



,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,11,Centro,Palacio,40.416514,-3.713708,5.0,Plaza,Tapas Restaurant,Spanish Restaurant,Bar,Vegetarian / Vegan Restaurant,Restaurant,Ice Cream Shop,Historic Site,Cocktail Bar,Beer Bar
1,12,Centro,Embajadores,40.409869,-3.702972,5.0,Hostel,Plaza,Hotel,Spanish Restaurant,Bar,Tapas Restaurant,Theater,Coffee Shop,Bookstore,Art Gallery
2,13,Centro,Cortes,40.415226,-3.697128,5.0,Restaurant,Plaza,Hotel,Café,Bar,Mediterranean Restaurant,Theater,Spanish Restaurant,Tapas Restaurant,Bookstore
3,14,Centro,Justicia,40.423660,-3.696469,5.0,Restaurant,Hotel,Spanish Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bookstore,Flower Shop,Deli / Bodega,Gay Bar
4,15,Centro,Universidad,40.407078,-3.709642,5.0,Bar,Tapas Restaurant,Theater,Plaza,Gym / Fitness Center,Art Gallery,Hotel,Italian Restaurant,Spanish Restaurant,Bistro


In [54]:
#Ignore cells with NaN
madrid_merged = madrid_merged.dropna()

# reset index after deleting rows
madrid_merged = madrid_merged.reset_index(drop=True)
madrid_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,11,Centro,Palacio,40.416514,-3.713708,5.0,Plaza,Tapas Restaurant,Spanish Restaurant,Bar,Vegetarian / Vegan Restaurant,Restaurant,Ice Cream Shop,Historic Site,Cocktail Bar,Beer Bar
1,12,Centro,Embajadores,40.409869,-3.702972,5.0,Hostel,Plaza,Hotel,Spanish Restaurant,Bar,Tapas Restaurant,Theater,Coffee Shop,Bookstore,Art Gallery
2,13,Centro,Cortes,40.415226,-3.697128,5.0,Restaurant,Plaza,Hotel,Café,Bar,Mediterranean Restaurant,Theater,Spanish Restaurant,Tapas Restaurant,Bookstore
3,14,Centro,Justicia,40.423660,-3.696469,5.0,Restaurant,Hotel,Spanish Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bookstore,Flower Shop,Deli / Bodega,Gay Bar
4,15,Centro,Universidad,40.407078,-3.709642,5.0,Bar,Tapas Restaurant,Theater,Plaza,Gym / Fitness Center,Art Gallery,Hotel,Italian Restaurant,Spanish Restaurant,Bistro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,211,Barajas,Alameda de Osuna,40.456636,-3.590373,5.0,Smoke Shop,Pizza Place,Bakery,Cocktail Bar,Plaza,Chinese Restaurant,Restaurant,Scenic Lookout,Shop & Service,Spanish Restaurant
123,212,Barajas,Aeropuerto,40.498348,-3.567781,5.0,Airport Lounge,Spanish Restaurant,Coffee Shop,Sporting Goods Shop,Breakfast Spot,Border Crossing,Airport Gate,Airport Service,Fast Food Restaurant,Duty-free Shop
124,213,Barajas,Casco Historico de Barajas,40.474305,-3.578636,5.0,Hotel,Spanish Restaurant,Restaurant,Tapas Restaurant,Japanese Restaurant,Bistro,Boarding House,Breakfast Spot,Brewery,Sandwich Place
125,214,Barajas,Timon,40.476295,-3.592266,1.0,Bakery,Women's Store,Fish & Chips Shop,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Escape Room


In [55]:
#Let´s visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_merged['Latitude'], madrid_merged['Longitude'], madrid_merged['Neighborhood'], madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
#Examine Clusters
#The discriminating venue categories that distinguish each cluster can be determined. 
#Depending on the defining categories, a specific name could be assigned to each cluster

In [57]:
#Cluster 0
madrid_merged.loc[madrid_merged['Cluster Labels'] == 0, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Retiro,0.0,Spanish Restaurant,Restaurant,Tapas Restaurant,Italian Restaurant,Seafood Restaurant,Gastropub,Mediterranean Restaurant,Bakery,Sandwich Place,Brewery
35,Tetuan,0.0,Spanish Restaurant,Gym / Fitness Center,Art Studio,Bookstore,Restaurant,Grocery Store,Park,Mediterranean Restaurant,Event Space,Exhibit
47,Fuencarral-El Pardo,0.0,Spanish Restaurant,Italian Restaurant,Pizza Place,Food & Drink Shop,Tapas Restaurant,Athletics & Sports,New American Restaurant,Park,Falafel Restaurant,Event Space
53,Moncloa-Aravaca,0.0,Spanish Restaurant,Soccer Field,Asian Restaurant,Supermarket,Women's Store,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
54,Moncloa-Aravaca,0.0,Spanish Restaurant,Bar,Arcade,Women's Store,Fish & Chips Shop,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
58,Latina,0.0,Spanish Restaurant,Grocery Store,Park,Supermarket,Paella Restaurant,Diner,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,Brewery
59,Latina,0.0,Pizza Place,Spanish Restaurant,Tapas Restaurant,Grocery Store,Asian Restaurant,Italian Restaurant,Women's Store,Falafel Restaurant,Event Space,Exhibit
74,Usera,0.0,Spanish Restaurant,Seafood Restaurant,Gastropub,Bar,Bakery,Fast Food Restaurant,BBQ Joint,Restaurant,Pub,Gym
76,Usera,0.0,Spanish Restaurant,Beer Garden,Athletics & Sports,Park,Fish & Chips Shop,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
77,Usera,0.0,Theater,Spanish Restaurant,Park,Beer Garden,Plaza,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Eastern European Restaurant


In [58]:
#Cluster 1
madrid_merged.loc[madrid_merged['Cluster Labels'] == 1, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
125,Barajas,1.0,Bakery,Women's Store,Fish & Chips Shop,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Escape Room


In [59]:
#Cluster 2
madrid_merged.loc[madrid_merged['Cluster Labels'] == 2, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Fuencarral-El Pardo,2.0,Canal Lock,Fast Food Restaurant,Escape Room,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Women's Store,Electronics Store


In [60]:
#Cluster 3
madrid_merged.loc[madrid_merged['Cluster Labels'] == 3, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
111,Vicalvaro,3.0,Mediterranean Restaurant,Women's Store,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Electronics Store


In [61]:
#Cluster 4
madrid_merged.loc[madrid_merged['Cluster Labels'] == 4, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Carabanchel,4.0,Bed & Breakfast,Park,Fast Food Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Women's Store


In [62]:
#Cluster 5
madrid_merged.loc[madrid_merged['Cluster Labels'] == 5, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,5.0,Plaza,Tapas Restaurant,Spanish Restaurant,Bar,Vegetarian / Vegan Restaurant,Restaurant,Ice Cream Shop,Historic Site,Cocktail Bar,Beer Bar
1,Centro,5.0,Hostel,Plaza,Hotel,Spanish Restaurant,Bar,Tapas Restaurant,Theater,Coffee Shop,Bookstore,Art Gallery
2,Centro,5.0,Restaurant,Plaza,Hotel,Café,Bar,Mediterranean Restaurant,Theater,Spanish Restaurant,Tapas Restaurant,Bookstore
3,Centro,5.0,Restaurant,Hotel,Spanish Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bookstore,Flower Shop,Deli / Bodega,Gay Bar
4,Centro,5.0,Bar,Tapas Restaurant,Theater,Plaza,Gym / Fitness Center,Art Gallery,Hotel,Italian Restaurant,Spanish Restaurant,Bistro
...,...,...,...,...,...,...,...,...,...,...,...,...
121,San Blas-Canillejas,5.0,Spanish Restaurant,Asian Restaurant,Hotel,Grocery Store,Snack Place,Restaurant,Tapas Restaurant,Café,Bistro,Breakfast Spot
122,Barajas,5.0,Smoke Shop,Pizza Place,Bakery,Cocktail Bar,Plaza,Chinese Restaurant,Restaurant,Scenic Lookout,Shop & Service,Spanish Restaurant
123,Barajas,5.0,Airport Lounge,Spanish Restaurant,Coffee Shop,Sporting Goods Shop,Breakfast Spot,Border Crossing,Airport Gate,Airport Service,Fast Food Restaurant,Duty-free Shop
124,Barajas,5.0,Hotel,Spanish Restaurant,Restaurant,Tapas Restaurant,Japanese Restaurant,Bistro,Boarding House,Breakfast Spot,Brewery,Sandwich Place


In [63]:
#Cluster 6
madrid_merged.loc[madrid_merged['Cluster Labels'] == 6, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
118,San Blas-Canillejas,6.0,Gastropub,Electronics Store,Women's Store,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fish & Chips Shop


In [64]:
#Cluster 7
madrid_merged.loc[madrid_merged['Cluster Labels'] == 7, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Carabanchel,7.0,Plaza,Pizza Place,Bar,Park,Falafel Restaurant,Escape Room,Event Space,Exhibit,Fabric Shop,Farmers Market
70,Carabanchel,7.0,Park,Pizza Place,Athletics & Sports,Metro Station,Fast Food Restaurant,Gym / Fitness Center,Women's Store,Falafel Restaurant,Escape Room,Event Space
78,Puente de Vallecas,7.0,Grocery Store,Business Service,Music Venue,Park,Gym / Fitness Center,Women's Store,Falafel Restaurant,Event Space,Exhibit,Fabric Shop
86,Moratalaz,7.0,Bar,Brewery,Park,Nightlife Spot,Dessert Shop,Women's Store,Fast Food Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market
91,Ciudad Lineal,7.0,Park,Metro Station,Supermarket,Bar,Fast Food Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Women's Store
103,Hortaleza,7.0,Grocery Store,Park,Soccer Field,Metro Station,Women's Store,Falafel Restaurant,Escape Room,Event Space,Exhibit,Fabric Shop
106,Villaverde,7.0,Plaza,Sports Bar,Grocery Store,Park,Falafel Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop
108,Villaverde,7.0,Spanish Restaurant,Bar,Grocery Store,Park,Women's Store,Farmers Market,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
115,San Blas-Canillejas,7.0,Snack Place,Park,Music Venue,Gym,Women's Store,Falafel Restaurant,Escape Room,Event Space,Exhibit,Fabric Shop
116,San Blas-Canillejas,7.0,Park,Food,Snack Place,Spanish Restaurant,Metro Station,Music Venue,Supermarket,Farmers Market,Event Space,Exhibit


In [65]:
#Cluster 8
madrid_merged.loc[madrid_merged['Cluster Labels'] == 8, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Latina,8.0,Golf Course,Garden,Pool,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Women's Store


In [66]:
#Cluster 9
madrid_merged.loc[madrid_merged['Cluster Labels'] == 9, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Moncloa-Aravaca,9.0,Trail,Women's Store,Eastern European Restaurant,Escape Room,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant


In [67]:
#Cluster 10
madrid_merged.loc[madrid_merged['Cluster Labels'] == 10, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Fuencarral-El Pardo,10.0,Golf Course,Soccer Field,Fast Food Restaurant,Escape Room,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Women's Store
90,Ciudad Lineal,10.0,Soccer Field,Pizza Place,Supermarket,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Eastern European Restaurant,Fast Food Restaurant


In [68]:
#Cluster 11
madrid_merged.loc[madrid_merged['Cluster Labels'] == 11, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,Moncloa-Aravaca,11.0,Italian Restaurant,Athletics & Sports,Gym Pool,Women's Store,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,11,Centro,Palacio,40.416514,-3.713708
1,12,Centro,Embajadores,40.409869,-3.702972
2,13,Centro,Cortes,40.415226,-3.697128
3,14,Centro,Justicia,40.423660,-3.696469
4,15,Centro,Universidad,40.407078,-3.709642


In [5]:
mynewdf.columns

Index(['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [7]:
mynewdf.dtypes

Postal Code       int64
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [8]:
print('The dataframe has {} Boroughs and {} Neighborhoods.'.format(
        len(mynewdf['Borough'].unique()),
        mynewdf.shape[0]
    )
)
mynewdf['Borough'].unique()

The dataframe has 21 Boroughs and 131 Neighborhoods.


array(['Centro', 'Arganzuela', 'Retiro', 'Salamanca', 'Chamartin',
       'Tetuan', 'Chamberi', 'Fuencarral-El Pardo', 'Moncloa-Aravaca',
       'Latina', 'Carabanchel', 'Usera', 'Puente de Vallecas',
       'Moratalaz', 'Ciudad Lineal', 'Hortaleza', 'Villaverde',
       'Villa de Vallecas', 'Vicalvaro', 'San Blas-Canillejas', 'Barajas'],
      dtype=object)

In [69]:
#import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#print('Libraries imported.')

Solving environment: done

# All requested packages already installed.



In [91]:
#Create a Map of Mexico City with Neighbourhoods superimposed using latitude and longitude values
#Passing Mexico City coordinates manually
latitude=19.42847
longitude=-99.12766

map_mexico = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(mynewdf['Latitude'], mynewdf['Longitude'], mynewdf['Borough'], mynewdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mexico)  
    
map_mexico

In [92]:
#Next, start utilizing the Foursquare API
CLIENT_ID = 'ET5MEYPN14RDTKJNHY4VVC0Q12R114CIOWGYVZ14FBSK1V32' # your Foursquare ID
CLIENT_SECRET = 'SUJMUIQO4X24ZKIPS31ZJFA12FJTFIC11SJQXSGZG4ZMRLSV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ET5MEYPN14RDTKJNHY4VVC0Q12R114CIOWGYVZ14FBSK1V32
CLIENT_SECRET:SUJMUIQO4X24ZKIPS31ZJFA12FJTFIC11SJQXSGZG4ZMRLSV


In [93]:
#Let´s explore the first neighborhood
mynewdf.loc[0, 'Neighborhood']
#Get the Neighborhood's Latitude and Longitude
neighborhood_latitude = mynewdf.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = mynewdf.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = mynewdf.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Col. San Jose Insurgentes are 19.366522099703925, -99.18519338197996.


In [96]:
#Now let´s get the top 100 venues in 500m distance
#Let´s create the GET request
#Import Latitude and Longitude Values
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id=ET5MEYPN14RDTKJNHY4VVC0Q12R114CIOWGYVZ14FBSK1V32&client_secret=SUJMUIQO4X24ZKIPS31ZJFA12FJTFIC11SJQXSGZG4ZMRLSV&ll=19.366522099703925,-99.18519338197996&oauth_token=&v=20180605&radius=500&limit=100'
url

'https://api.foursquare.com/v2/venues/explore?client_id=ET5MEYPN14RDTKJNHY4VVC0Q12R114CIOWGYVZ14FBSK1V32&client_secret=SUJMUIQO4X24ZKIPS31ZJFA12FJTFIC11SJQXSGZG4ZMRLSV&ll=19.366522099703925,-99.18519338197996&oauth_token=&v=20180605&radius=500&limit=100'

In [97]:
#Send the GET request ans examine the result
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6010afc7d60cca30f68c438a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'San José Insurgentes',
  'headerFullLocation': 'San José Insurgentes, Mexico City',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 38,
  'suggestedBounds': {'ne': {'lat': 19.37102210420393,
    'lng': -99.18043238606482},
   'sw': {'lat': 19.36202209520392, 'lng': -99.18995437789509}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d7d435e8f89224b6d86b326',
       'name': 'Parque de los Insurgentes',
       'location': {'address': 'Av. Plateros',
        'crossStreet': 'Mercaderes',
        'lat': 19.36615254318577,
        'lng': -99.18336993467477,


In [98]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [99]:
#Clean the json and structure into a Pandas Dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Parque de los Insurgentes,Park,19.366153,-99.183370
1,Central Onceveintitres,Antique Shop,19.365681,-99.183898
2,La Taperia,Tapas Restaurant,19.365214,-99.184934
3,Santa Clara,Ice Cream Shop,19.363769,-99.185129
4,Jardin Interior,Vegetarian / Vegan Restaurant,19.364073,-99.185962


In [100]:
#How many venues returned by FourSquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

38 venues were returned by Foursquare.


In [ ]:
## Next Explore Neighbourhoods in Mexico City

In [103]:
#Create a function to repeat the same process to all Neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
               
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [106]:
#Run the above function on each Neighbourhood and create the Dataframe toronto_venues
mexico_venues = getNearbyVenues(names=mynewdf['Neighborhood'],
                                   latitudes=mynewdf['Latitude'],
                                   longitudes=mynewdf['Longitude'])

Col. San Jose Insurgentes
Col. Las Palmas
Col. Centro de Ciudad Santa Fe
Col. Lomas de San Angel Inn
Fracc. Industrial de Vallejo
Col. La Preciosa
Col. Coltongo
Col. Santa Cruz Acayucan
Col. Nueva Santa Maria
Col. San Salvador Xochimanca
Col. Narvarte Poniente
Col. Narvarte Oriente
Col. Alamos
Col. Del Valle
Col. Napoles
Col. Nonoalco
Col. Moderna
Col. San Jose Insurgentes
Col. Credito Constructor
Col. del Carmen
Col. San Pablo Tepetlapa
Col. Pueblo de Santa Ursula Coapa
Col. Santa Ursula Coapa
Col. Lomas de Vista Hermosa
Col. Bosques de las Lomas
Col. Centro
Col. Juarez
Col. Roma
Col. Doctores
Col. Obrera
Col. Nonoalco Tlatelolco
Col. Maximimo Avila Camacho
Col. Heroe de Nacozari
Col. Industrial
Col. Tres Estrellas
Col. San Juan de Aragon
Col. Pantitlan
Col. Viaducto Piedad
Col. Santa Anita
Col. Granjas Mexico
Col. Agricola Oriental
Col. Central de Abastos
Col. Sector Popular
Col. Granjas San Antonio
Col. Ejercito de Oriente
Col. Santa Martha Acatitla
Col. Lomas de Zaragoza
Col. Xalpa

In [107]:
#Size of the resulting Dataframe
print(mexico_venues.shape)
mexico_venues.head()


(2867, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Col. San Jose Insurgentes,19.366522,-99.185193,Parque de los Insurgentes,19.366153,-99.183370,Park
1,Col. San Jose Insurgentes,19.366522,-99.185193,Central Onceveintitres,19.365681,-99.183898,Antique Shop
2,Col. San Jose Insurgentes,19.366522,-99.185193,La Taperia,19.365214,-99.184934,Tapas Restaurant
3,Col. San Jose Insurgentes,19.366522,-99.185193,Santa Clara,19.363769,-99.185129,Ice Cream Shop
4,Col. San Jose Insurgentes,19.366522,-99.185193,Jardin Interior,19.364073,-99.185962,Vegetarian / Vegan Restaurant


In [108]:
#Find how many unique categories can be curated from all the returned venues
print('There are {} unique categories.'.format(len(mexico_venues['Venue Category'].unique())))

There are 280 unique categories.


In [110]:
##Analyze each Neighborhood
# one hot encoding
mexico_onehot = pd.get_dummies(mexico_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mexico_onehot['Neighborhood'] = mexico_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mexico_onehot.columns[-1]] + list(mexico_onehot.columns[:-1])
mexico_onehot = mexico_onehot[fixed_columns]

mexico_onehot.head()

,Yoga Studio,ATM,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College History Building,College Theater,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distillery,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Radio Station,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Road,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sausage Shop,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Student Center,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Tree,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Warehouse Store,Watch Shop,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [111]:
#New dataframe size
mexico_onehot.shape

(2867, 280)

In [112]:
##Let´s group rows by Neighborhood and by taking the mean of the frequency of the ocurrence of each category
mexico_grouped = mexico_onehot.groupby('Neighborhood').mean().reset_index()
mexico_grouped


,Neighborhood,Yoga Studio,ATM,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College History Building,College Theater,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distillery,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Radio Station,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Road,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sausage Shop,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Student Center,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Tree,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Warehouse Store,Watch Shop,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,Col. Agricola Oriental,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [113]:
#Confirm new size
mexico_grouped.shape


(73, 280)

In [114]:
#Let´s print each neighbourhood along with the top 5 most common venues
num_top_venues = 5

for hood in mexico_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mexico_grouped[mexico_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----Col. Agricola Oriental----
                venue  freq
0  Mexican Restaurant  0.21
1          Taco Place  0.12
2          Restaurant  0.08
3      Clothing Store  0.04
4  Seafood Restaurant  0.04


----Col. Alamos----
                venue  freq
0  Mexican Restaurant  0.13
1          Taco Place  0.07
2      Ice Cream Shop  0.05
3              Bakery  0.05
4  Seafood Restaurant  0.05


----Col. America----
                    venue  freq
0              Taco Place  0.17
1  Furniture / Home Store  0.08
2      Mexican Restaurant  0.08
3              Restaurant  0.08
4             Coffee Shop  0.08


----Col. Anzures----
                venue  freq
0         Coffee Shop  0.06
1          Restaurant  0.06
2  Mexican Restaurant  0.05
3        Liquor Store  0.04
4          Steakhouse  0.04


----Col. Argentina Poniente----
                venue  freq
0  Mexican Restaurant   0.2
1      Farmers Market   0.2
2            Pharmacy   0.1
3              Bakery   0.1
4      Shop & Service   0.1


-

In [115]:
#Put into a Pandas Dataframe
#First use function to classify venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [116]:
#Now let´s create a dataframe and display the top 10 venues for each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mexico_grouped['Neighborhood']

for ind in np.arange(mexico_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mexico_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Col. Agricola Oriental,Mexican Restaurant,Taco Place,Restaurant,Clothing Store,Gym Pool,Snack Place,Food,Seafood Restaurant,Fast Food Restaurant,Food Truck
1,Col. Alamos,Mexican Restaurant,Taco Place,Bakery,Coffee Shop,Ice Cream Shop,Seafood Restaurant,Restaurant,Gym / Fitness Center,Diner,Café
2,Col. America,Taco Place,Restaurant,Furniture / Home Store,Coffee Shop,Mexican Restaurant,Pharmacy,Bakery,Outdoor Sculpture,Garden,Bar
3,Col. Anzures,Coffee Shop,Restaurant,Mexican Restaurant,Café,Spa,Liquor Store,Steakhouse,Japanese Restaurant,Health & Beauty Service,Italian Restaurant
4,Col. Argentina Poniente,Mexican Restaurant,Farmers Market,History Museum,Bakery,Shop & Service,Shopping Mall,Pharmacy,Ice Cream Shop,Food & Drink Shop,Food


In [118]:
## Cluster Neighbourhoods
# Run k-means to cluster the neighbourhoods into 7 clusters
# set number of clusters
kclusters = 12

mexico_grouped_clustering = mexico_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mexico_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([ 0, 10, 10,  3,  8,  3, 10, 10,  0,  3, 10, 10, 10,  4, 10,  3,  0,
        3,  0, 10, 10, 10, 10,  3, 10,  0,  0, 10, 10,  3,  0,  6, 10,  0,
       10, 10,  0,  3, 10, 10, 10, 10, 10,  0, 10, 10,  0,  3,  3, 11, 10,
        3,  0,  0, 10, 10,  3,  0,  0,  9,  0,  0,  7, 11,  0, 10, 10,  0,
        2, 10,  5,  1, 10], dtype=int32)

In [119]:
#Let´s create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mexico_merged = mynewdf

# merge mexico_grouped with mynewdf to add latitude/longitude for each neighborhood
mexico_merged = mexico_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

mexico_merged.head() # check the last columns!



,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1020,Alvaro Obregon,Col. San Jose Insurgentes,19.366522,-99.185193,10,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Tapas Restaurant,Italian Restaurant,Bakery,Seafood Restaurant,Shopping Mall,Taco Place,Farmers Market
1,1110,Alvaro Obregon,Col. Las Palmas,19.398938,-99.211254,0,Mexican Restaurant,Taco Place,Convenience Store,Cafeteria,Stables,Music Venue,Sushi Restaurant,Gym,Coffee Shop,Gym Pool
2,1210,Alvaro Obregon,Col. Centro de Ciudad Santa Fe,19.382992,-99.241860,0,Mexican Restaurant,Burger Joint,Taco Place,Record Shop,Restaurant,Flea Market,Laundromat,Farmers Market,Pizza Place,Seafood Restaurant
3,1790,Alvaro Obregon,Col. Lomas de San Angel Inn,19.345009,-99.205693,10,Taco Place,Café,Garden Center,Restaurant,Park,Music Venue,Bakery,Swim School,Seafood Restaurant,Student Center
4,2300,Azcapotzalco,Fracc. Industrial de Vallejo,19.496314,-99.163649,1,Rental Car Location,Food Truck,Women's Store,Food & Drink Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food


In [120]:
#Ignore cells with NaN
mexico_merged = mexico_merged.dropna()

# reset index after deleting rows
mexico_merged = mexico_merged.reset_index(drop=True)
mexico_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1020,Alvaro Obregon,Col. San Jose Insurgentes,19.366522,-99.185193,10,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Tapas Restaurant,Italian Restaurant,Bakery,Seafood Restaurant,Shopping Mall,Taco Place,Farmers Market
1,1110,Alvaro Obregon,Col. Las Palmas,19.398938,-99.211254,0,Mexican Restaurant,Taco Place,Convenience Store,Cafeteria,Stables,Music Venue,Sushi Restaurant,Gym,Coffee Shop,Gym Pool
2,1210,Alvaro Obregon,Col. Centro de Ciudad Santa Fe,19.382992,-99.241860,0,Mexican Restaurant,Burger Joint,Taco Place,Record Shop,Restaurant,Flea Market,Laundromat,Farmers Market,Pizza Place,Seafood Restaurant
3,1790,Alvaro Obregon,Col. Lomas de San Angel Inn,19.345009,-99.205693,10,Taco Place,Café,Garden Center,Restaurant,Park,Music Venue,Bakery,Swim School,Seafood Restaurant,Student Center
4,2300,Azcapotzalco,Fracc. Industrial de Vallejo,19.496314,-99.163649,1,Rental Car Location,Food Truck,Women's Store,Food & Drink Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food
5,2460,Azcapotzalco,Col. La Preciosa,19.486791,-99.193973,0,Mexican Restaurant,Taco Place,Bakery,BBQ Joint,Burger Joint,Ice Cream Shop,Burrito Place,Gym,Market,Lounge
6,2630,Azcapotzalco,Col. Coltongo,19.483454,-99.152090,3,Clothing Store,Seafood Restaurant,Ice Cream Shop,Mexican Restaurant,Fast Food Restaurant,Hotel,Pizza Place,Sporting Goods Shop,Coffee Shop,Tea Room
7,2770,Azcapotzalco,Col. Santa Cruz Acayucan,19.475747,-99.194386,0,Mexican Restaurant,Taco Place,BBQ Joint,Cosmetics Shop,Market,Seafood Restaurant,Big Box Store,Baseball Field,Café,Burger Joint
8,2800,Azcapotzalco,Col. Nueva Santa Maria,19.462959,-99.168442,10,Mexican Restaurant,Ice Cream Shop,Taco Place,Café,Bakery,Sushi Restaurant,Restaurant,Park,Coffee Shop,Snack Place
9,2870,Azcapotzalco,Col. San Salvador Xochimanca,19.459971,-99.173995,9,Mexican Restaurant,Gastropub,Pet Store,Mediterranean Restaurant,Gym,Gym / Fitness Center,Tennis Court,Diner,Food Truck,Food & Drink Shop


In [121]:
#Let´s visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mexico_merged['Latitude'], mexico_merged['Longitude'], mexico_merged['Neighborhood'], mexico_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
#Examine Clusters
#The discriminating venue categories that distinguish each cluster can be determined. 
#Depending on the defining categories, a specific name could be assigned to each cluster

In [122]:
#Cluster 0
mexico_merged.loc[mexico_merged['Cluster Labels'] == 0, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Alvaro Obregon,0,Mexican Restaurant,Taco Place,Convenience Store,Cafeteria,Stables,Music Venue,Sushi Restaurant,Gym,Coffee Shop,Gym Pool
2,Alvaro Obregon,0,Mexican Restaurant,Burger Joint,Taco Place,Record Shop,Restaurant,Flea Market,Laundromat,Farmers Market,Pizza Place,Seafood Restaurant
5,Azcapotzalco,0,Mexican Restaurant,Taco Place,Bakery,BBQ Joint,Burger Joint,Ice Cream Shop,Burrito Place,Gym,Market,Lounge
7,Azcapotzalco,0,Mexican Restaurant,Taco Place,BBQ Joint,Cosmetics Shop,Market,Seafood Restaurant,Big Box Store,Baseball Field,Café,Burger Joint
16,Benito Juarez,0,Mexican Restaurant,Taco Place,Sushi Restaurant,Bar,Bakery,Gym,Café,Chinese Restaurant,Restaurant,Park
20,Coyoacan,0,Taco Place,Museum,Pizza Place,Bakery,Mexican Restaurant,Restaurant,Art Museum,Burrito Place,Sports Club,Soup Place
29,Cuauhtemoc,0,Taco Place,Bar,Seafood Restaurant,Mexican Restaurant,Bakery,Indie Theater,Gym,Fried Chicken Joint,Café,Stationery Store
35,Gustavo A. Madero,0,Mexican Restaurant,Taco Place,Pharmacy,Lounge,Shop & Service,Food Truck,Market,Seafood Restaurant,Fish & Chips Shop,Other Great Outdoors
37,Iztacalco,0,Mexican Restaurant,Juice Bar,Taco Place,Ice Cream Shop,Gym / Fitness Center,Seafood Restaurant,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Gourmet Shop
38,Iztacalco,0,Mexican Restaurant,Taco Place,Food Truck,Restaurant,Ice Cream Shop,Farmers Market,Breakfast Spot,Motorcycle Shop,Motel,Seafood Restaurant


In [125]:
#Cluster 1
mexico_merged.loc[mexico_merged['Cluster Labels'] == 1, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Azcapotzalco,1,Rental Car Location,Food Truck,Women's Store,Food & Drink Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food


In [126]:
#Cluster 2
mexico_merged.loc[mexico_merged['Cluster Labels'] == 2, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Delegacion Iztapalapa,2,Bakery,Coffee Shop,Arcade,Food & Drink Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food


In [127]:
#Cluster 3
mexico_merged.loc[mexico_merged['Cluster Labels'] == 3, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Azcapotzalco,3,Clothing Store,Seafood Restaurant,Ice Cream Shop,Mexican Restaurant,Fast Food Restaurant,Hotel,Pizza Place,Sporting Goods Shop,Coffee Shop,Tea Room
14,Benito Juarez,3,Coffee Shop,Mexican Restaurant,Seafood Restaurant,Pizza Place,Japanese Restaurant,Café,BBQ Joint,Taco Place,Ice Cream Shop,Flea Market
23,Cuajimalpa,3,Gym,Coffee Shop,Restaurant,Deli / Bodega,Electronics Store,Nightclub,Bus Stop,Cafeteria,Music School,Café
24,Cuajimalpa,3,Pizza Place,Taco Place,Mexican Restaurant,Yoga Studio,Scenic Lookout,Coffee Shop,Hotel,Dance Studio,Athletics & Sports,Piano Bar
26,Cuauhtemoc,3,Coffee Shop,Bakery,Hotel,Pizza Place,Chinese Restaurant,Cosmetics Shop,Seafood Restaurant,Restaurant,Sculpture Garden,Men's Store
27,Cuauhtemoc,3,Coffee Shop,Boutique,Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Tea Room,Sandwich Place,Pizza Place,Mexican Restaurant,Health & Beauty Service
39,Iztacalco,3,Sports Club,Stadium,Racetrack,Athletics & Sports,Music Venue,Gym,Lounge,Coffee Shop,Circus,Rest Area
49,Delegacion Iztapalapa,3,Gym,Food Truck,Ice Cream Shop,Deli / Bodega,Grocery Store,Print Shop,Dessert Shop,Snack Place,Pizza Place,Fondue Restaurant
57,Miguel Hidalgo,3,Restaurant,Café,Convenience Store,Bakery,Mexican Restaurant,Gym,Brewery,Italian Restaurant,Coffee Shop,Sandwich Place
58,Miguel Hidalgo,3,Seafood Restaurant,Coffee Shop,Bakery,Café,Italian Restaurant,Ice Cream Shop,Bridal Shop,Gym / Fitness Center,Furniture / Home Store,Sandwich Place


In [128]:
#Cluster 4
mexico_merged.loc[mexico_merged['Cluster Labels'] == 4, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Delegacion Iztapalapa,4,Wings Joint,Ice Cream Shop,Fast Food Restaurant,Flea Market,Mexican Restaurant,Forest,Food Truck,Food Stand,Food Service,Farmers Market


In [129]:
#Cluster 5
mexico_merged.loc[mexico_merged['Cluster Labels'] == 5, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Tlahuac,5,Martial Arts School,Flea Market,Pool Hall,Taco Place,Women's Store,Food,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Fondue Restaurant


In [130]:
#Cluster 6
mexico_merged.loc[mexico_merged['Cluster Labels'] == 6, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,Venustiano Carranza,6,Taco Place,Mexican Restaurant,Seafood Restaurant,Restaurant,Lounge,Convenience Store,Snack Place,Park,Argentinian Restaurant,Fondue Restaurant


In [131]:
#Cluster 7
mexico_merged.loc[mexico_merged['Cluster Labels'] == 7, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Delegacion Iztapalapa,7,Mobile Phone Shop,Recreation Center,Food Service,Gym,Taco Place,Plaza,Seafood Restaurant,Pizza Place,Women's Store,Farmers Market


In [133]:
#Cluster 8
mexico_merged.loc[mexico_merged['Cluster Labels'] == 8, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Miguel Hidalgo,8,Mexican Restaurant,Farmers Market,History Museum,Bakery,Shop & Service,Shopping Mall,Pharmacy,Ice Cream Shop,Food & Drink Shop,Food


In [134]:
#Cluster 9
mexico_merged.loc[mexico_merged['Cluster Labels'] == 9, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Azcapotzalco,9,Mexican Restaurant,Gastropub,Pet Store,Mediterranean Restaurant,Gym,Gym / Fitness Center,Tennis Court,Diner,Food Truck,Food & Drink Shop


In [135]:
#Cluster 10
mexico_merged.loc[mexico_merged['Cluster Labels'] == 10, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alvaro Obregon,10,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Tapas Restaurant,Italian Restaurant,Bakery,Seafood Restaurant,Shopping Mall,Taco Place,Farmers Market
3,Alvaro Obregon,10,Taco Place,Café,Garden Center,Restaurant,Park,Music Venue,Bakery,Swim School,Seafood Restaurant,Student Center
8,Azcapotzalco,10,Mexican Restaurant,Ice Cream Shop,Taco Place,Café,Bakery,Sushi Restaurant,Restaurant,Park,Coffee Shop,Snack Place
10,Benito Juarez,10,Mexican Restaurant,Taco Place,Coffee Shop,Café,Bakery,Ice Cream Shop,Pizza Place,Restaurant,Food Truck,Gym / Fitness Center
11,Benito Juarez,10,Taco Place,Mexican Restaurant,Coffee Shop,Burger Joint,Café,BBQ Joint,Optical Shop,Sushi Restaurant,Supermarket,Flea Market
12,Benito Juarez,10,Mexican Restaurant,Taco Place,Bakery,Coffee Shop,Ice Cream Shop,Seafood Restaurant,Restaurant,Gym / Fitness Center,Diner,Café
13,Benito Juarez,10,Mexican Restaurant,Spa,Pharmacy,Grocery Store,Convenience Store,Italian Restaurant,Gym / Fitness Center,Salon / Barbershop,Fried Chicken Joint,Department Store
15,Benito Juarez,10,Mexican Restaurant,Taco Place,Coffee Shop,Gym / Fitness Center,Furniture / Home Store,Food Truck,Bar,Convenience Store,Donut Shop,Pet Store
17,Benito Juarez,10,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Tapas Restaurant,Italian Restaurant,Bakery,Seafood Restaurant,Shopping Mall,Taco Place,Farmers Market
18,Benito Juarez,10,Mexican Restaurant,Coffee Shop,Restaurant,Café,Park,Tapas Restaurant,Seafood Restaurant,French Restaurant,Argentinian Restaurant,Steakhouse


In [136]:
#Cluster 11
mexico_merged.loc[mexico_merged['Cluster Labels'] == 11, mexico_merged.columns[[1] + list(range(5, mexico_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Coyoacan,11,Taco Place,Office,Motel,Flower Shop,Salad Place,Auto Garage,Campground,Mexican Restaurant,Convenience Store,Creperie
22,Coyoacan,11,Office,Motel,Taco Place,Flower Shop,Salad Place,Auto Garage,Campground,Mexican Restaurant,Convenience Store,Creperie
